In [17]:
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq
import os
from langchain_huggingface import HuggingFaceEmbeddings

In [14]:
from key import GROQ_API_KEY

In [15]:
#!pip install PyPDF2

## Fuck You Open AI Embeddings

In [16]:
# !pip install langchain-huggingface


  Obtaining dependency information for langchain-huggingface from https://files.pythonhosted.org/packages/9d/f8/77a303ddc492f6eed8bf0979f2bc6db4fa6eb1089c5e9f0f977dd87bc9c2/langchain_huggingface-0.1.2-py3-none-any.whl.metadata
  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Obtaining dependency information for sentence-transformers>=2.6.0 from https://files.pythonhosted.org/packages/8b/c8/990e22a465e4771338da434d799578865d6d7ef1fdb50bd844b7ecdcfa19/sentence_transformers-3.3.1-py3-none-any.whl.metadata
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)


Using cached langchain_huggingface-0.1.2-py3-none-any.whl (21 kB)
Using cached sentence_transformers-3.3.1-py3-none-any.whl (268 kB)


In [22]:
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/e4/9c/aed8b7c6c490c777c404131b3f6a68e4924fbc149620dc6d6a3563435371/faiss_cpu-1.9.0.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 974.0 kB/s eta 0:00:00m eta 0:00:010:00:01


In [23]:

class PDFChatbot:
    def __init__(self, api_key):
        """
        Initialize the chatbot with your OpenAI API key
        """
        os.environ["GROQ_API_KEY"] = api_key
        self.chat_history = []
        self.chain = None
        
    def load_pdf(self, pdf_path):
        """
        Load and extract text from a PDF file
        """
        text = ""
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text
    
    def process_pdf(self, pdf_path):
        """
        Process PDF content and create a conversational chain
        """
        # Extract text from PDF
        raw_text = self.load_pdf(pdf_path)
#         # Replacing \n and others
#         raw_text.replace
        # Split text into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )
        texts = text_splitter.split_text(raw_text)
        
        # Create embeddings and vector store
        #embeddings = OpenAIEmbeddings()
        
        embeddings_model = HuggingFaceEmbeddings()
        vectorstore = FAISS.from_texts(texts, embeddings_model)
        
        # Create the conversational chain
        self.chain = ConversationalRetrievalChain.from_llm(
            llm=ChatGroq(model="llama3-8b-8192"),
            retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
        )
        
        return {"raw_text":raw_text,"text_splitted":texts}
        
    def ask_question(self, question):
        """
        Ask a question about the PDF content
        """
        if not self.chain:
            return "Please load a PDF first using process_pdf()"
        
        response = self.chain({"question": question, "chat_history": self.chat_history})
        self.chat_history.append((question, response['answer']))
        
        return response['answer']




    
    # Ask questions
    while True:
        question = input("\nAsk a question about the PDF (or 'quit' to exit): ")
        if question.lower() == 'quit':
            break
        
        answer = chatbot.ask_question(question)
        print("\nAnswer:", answer)



In [21]:
chatbot = PDFChatbot(GROQ_API_KEY)
    
# Process a PDF file
chatbot.process_pdf("./sample_test.pdf")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [24]:
chatbot = PDFChatbot(GROQ_API_KEY)
    
# Process a PDF file
value=chatbot.process_pdf("./sample_test.pdf")

value

{'raw_text': ' \nDepartment of English  \nMidterm  Examination Schedule  (Summer -2024)  \n \n \nBA (Honors) 1st Semester  \nDate  Day Course Code  Course Title  Time  \n29.09.2024  Sunday  ENG 1101  Listening and Speaking  \n03:00pm -04:00p m 01.10.2024  Tuesday  ENG 1102  Reading  \n03.10.2024  Thursday  ENG 1103  Writing  \n06.10.2024  Sunday  ENG 1104  Introduction to Poe try \n08.10.2024  Tuesday  ENG 1105  Introduction to Fiction  \n \nBA (Honors) 2nd Semester  \nDate  Day Course Code  Course Title  Time  \n29.09.2024  Sunday  ENG 1201  Introduction to Drama  \n10:00am -11:00am  01.10.2024  Tuesday  ENG 1202  Introduction to Non Fiction  \n03.10. 2024  Thursday  ENG 1203  History of England  \n06.10.2024  Sunday  ENG 1204  Academic Writing  \n07.10.2024  Monday  BAN 1001  History of the Emergence of Bangladesh  12:00pm -01:00pm  \n08.10.2024  Tuesday  CSE 1003  Computer Fundamentals  10:00am -11:00am  \n \nBA (Honors) 3rd Semester  \nDate  Day Course Code  Course Title  Time  \n2

In [26]:
print(value["text_splitted"])

['Department of English  \nMidterm  Examination Schedule  (Summer -2024)  \n \n \nBA (Honors) 1st Semester  \nDate  Day Course Code  Course Title  Time  \n29.09.2024  Sunday  ENG 1101  Listening and Speaking  \n03:00pm -04:00p m 01.10.2024  Tuesday  ENG 1102  Reading  \n03.10.2024  Thursday  ENG 1103  Writing  \n06.10.2024  Sunday  ENG 1104  Introduction to Poe try \n08.10.2024  Tuesday  ENG 1105  Introduction to Fiction  \n \nBA (Honors) 2nd Semester  \nDate  Day Course Code  Course Title  Time  \n29.09.2024  Sunday  ENG 1201  Introduction to Drama  \n10:00am -11:00am  01.10.2024  Tuesday  ENG 1202  Introduction to Non Fiction  \n03.10. 2024  Thursday  ENG 1203  History of England  \n06.10.2024  Sunday  ENG 1204  Academic Writing  \n07.10.2024  Monday  BAN 1001  History of the Emergence of Bangladesh  12:00pm -01:00pm  \n08.10.2024  Tuesday  CSE 1003  Computer Fundamentals  10:00am -11:00am  \n \nBA (Honors) 3rd Semester  \nDate  Day Course Code  Course Title  Time', '08.10.2024  Tues

In [27]:
chatbot.ask_question("Hi")

/tmp/ipykernel_10127/715666710.py:57: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.chain({"question": question, "chat_history": self.chat_history})


"Hi! It seems you're a student at the Department of English, and you have a course schedule for the summer semester of 2024. Which course or topic would you like to know more about or discuss? I'm here to help!"

In [30]:
chatbot.ask_question("ENG 1105 course exam date and time")

'According to the provided schedule, the exam date and time for the ENG 1105 course, "Introduction to Fiction", is:\n\n* Date: 08.10.2024 (Tuesday)\n* Time: Not specified in the provided schedule (note that the schedule only provides the time for ENG 1104, "Introduction to Poetry", which is 03:00pm -04:00pm, but does not provide the time for ENG 1105)'

In [31]:
chatbot.ask_question("History of the Emergence of Bangladesh course exam date and time")

'To answer this question, I would need to know which course you are referring to. The schedule provided lists multiple courses and their corresponding dates and times. Could you please specify the course code and title of the course for which you want to know the exam date and time?'

In [32]:
chatbot.ask_question("BAN 1001")

'According to the given schedule, the exam date and time for the BAN 1001 course are:\n\nDate: 07.10.2024 (Monday)\nTime: 12:00pm -01:00pm'

In [33]:
# !pip freeze > requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
